In [31]:
import numpy as np
import torch
import torch.nn as nn

from model import SegmentClassifier
from estimator import Estimator
from graph import load_graphs, SparseGraph, graph_from_sparse
feature_scale = np.array([1000., np.pi/8, 1000.])
n_features = feature_scale.shape[0]

cuda = False

In [32]:
if cuda:
    np_to_torch = lambda x, volatile=False, dtype=np.float32: (torch.tensor(x.astype(dtype), requires_grad=False).cuda())
else:
    np_to_torch = lambda x, volatile=False, dtype=np.float32: (torch.tensor(x.astype(dtype), requires_grad=False))

if cuda:
    torch_to_np = lambda x: x.cpu().numpy()
else:
    torch_to_np = lambda x: x.detach().numpy()

In [33]:
hidden_dim = 4
n_iters = 1
model = SegmentClassifier(input_dim=n_features, hidden_dim=hidden_dim, n_iters=n_iters)
estim = Estimator(model, loss_func=nn.BCELoss(), cuda=cuda, l1= 0)
#estim.load_checkpoint('small_model/model_3_1iteration.pt')
estim.load_checkpoint('small_model/model_1iteration.pt')

2018-08-20 14:55:08.105840 Model: 
SegmentClassifier(
  (input_network): Sequential(
    (0): Linear(in_features=3, out_features=4, bias=True)
    (1): Tanh()
  )
  (edge_network): EdgeNetwork(
    (network): Sequential(
      (0): Linear(in_features=14, out_features=4, bias=True)
      (1): Tanh()
      (2): Linear(in_features=4, out_features=1, bias=True)
      (3): Sigmoid()
    )
  )
  (node_network): NodeNetwork(
    (network): Sequential(
      (0): Linear(in_features=21, out_features=4, bias=True)
      (1): Tanh()
      (2): Linear(in_features=4, out_features=4, bias=True)
      (3): Tanh()
    )
  )
)
2018-08-20 14:55:08.106136 Parameters: 189


In [34]:
#graph = load_graphs(['small_model_10graphs_2/graph000001.npz'], SparseGraph)
graph = load_graphs(['small_model_graphs/graph000001.npz'], SparseGraph)

g = graph[0]
g = graph_from_sparse(g)

X = np_to_torch(g.X, volatile=False)
Ri = np_to_torch(g.Ri, volatile=False) 
Ro = np_to_torch(g.Ro, volatile=False)

X = X.view(1, X.shape[0], X.shape[1])
Ri = Ri.view(1, Ri.shape[0], Ri.shape[1])
Ro = Ro.view(1, Ro.shape[0], Ro.shape[1])
edge_net = model.forward([X, Ri, Ro]) 
print(edge_net)

torch.Size([1, 4, 3])
torch.Size([1, 4, 7])
torch.Size([1, 4, 4])
torch.Size([1, 4, 7])
torch.Size([1, 4, 4])
torch.Size([1, 4, 7])
tensor([[ 0.9993,  0.0009,  0.0009,  0.9997]])
